Topic 28 - Project - Training from Scratch vs Transfer Learning (100% Completed)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
X_train_full = X_train_full[:30000]
y_train_full = y_train_full[:30000]

In [4]:
X_test = X_test[:5000]
y_test = y_test[:5000]

In [5]:
X_train_full = X_train_full / 255
X_test = X_test / 255

In [6]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
def split_dataset(X, y):
    y_5_or_6 = (y==5) | (y==6)
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2
    y_B = (y[y_5_or_6] == 6).astype(np.float32)
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

In [8]:
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)

In [9]:
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)

In [10]:
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)

In [11]:
tf.random.set_seed(42)
np.random.seed(42)

In [12]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [17]:
sgd = keras.optimizers.SGD(learning_rate=1e-3)

In [18]:
model_A.compile(loss="sparse_categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [19]:
history = model_A.fit(X_train_A, y_train_A, epochs=5, validation_data=(X_valid_A, y_valid_A))

Epoch 1/5
622/622 [==============================] - 3s 4ms/step - loss: 0.3108 - accuracy: 0.8909 - val_loss: 0.3083 - val_accuracy: 0.8944
Epoch 2/5
622/622 [==============================] - 2s 4ms/step - loss: 0.2999 - accuracy: 0.8961 - val_loss: 0.3081 - val_accuracy: 0.8949
Epoch 3/5
622/622 [==============================] - 2s 4ms/step - loss: 0.2917 - accuracy: 0.8983 - val_loss: 0.3208 - val_accuracy: 0.8891
Epoch 4/5
622/622 [==============================] - 2s 4ms/step - loss: 0.2841 - accuracy: 0.9024 - val_loss: 0.2867 - val_accuracy: 0.9056
Epoch 5/5
622/622 [==============================] - 2s 4ms/step - loss: 0.2776 - accuracy: 0.9038 - val_loss: 0.2882 - val_accuracy: 0.8996


In [21]:
model_A.save("my_model_A.h5")

OSError: Unable to create file (unable to truncate a file which is already open)

In [24]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="softmax"))

In [25]:
model_B.compile(loss="binary_crossentropy", optimizer=sgd, metrics="accuracy")

In [26]:
history = model_B.fit(X_train_B, y_train_B, epochs=5, validation_data=(X_valid_B, y_valid_B))

Epoch 1/5
161/161 [==============================] - 1s 6ms/step - loss: 0.2678 - accuracy: 0.5050 - val_loss: 0.1301 - val_accuracy: 0.5000
Epoch 2/5
161/161 [==============================] - 1s 4ms/step - loss: 0.0910 - accuracy: 0.5050 - val_loss: 0.0765 - val_accuracy: 0.5000
Epoch 3/5
161/161 [==============================] - 1s 4ms/step - loss: 0.0573 - accuracy: 0.5050 - val_loss: 0.0565 - val_accuracy: 0.5000
Epoch 4/5
161/161 [==============================] - 1s 5ms/step - loss: 0.0423 - accuracy: 0.5050 - val_loss: 0.0456 - val_accuracy: 0.5000
Epoch 5/5
161/161 [==============================] - 1s 4ms/step - loss: 0.0335 - accuracy: 0.5050 - val_loss: 0.0389 - val_accuracy: 0.5000


In [29]:
model_B.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

In [30]:
model_A_clone = keras.models.clone_model(model_A)

In [31]:
model_A_clone.set_weights(model_A.get_weights())

In [32]:
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])

In [33]:
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [35]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

In [36]:
model_B_on_A.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                

In [38]:
model_B_on_A.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=1e-3), metrics=["accuracy"])

In [39]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=5, validation_data=(X_valid_B, y_valid_B))

Epoch 1/5
161/161 [==============================] - 1s 4ms/step - loss: 0.3452 - accuracy: 0.8537 - val_loss: 0.2469 - val_accuracy: 0.9351
Epoch 2/5
161/161 [==============================] - 0s 3ms/step - loss: 0.1901 - accuracy: 0.9604 - val_loss: 0.1624 - val_accuracy: 0.9696
Epoch 3/5
161/161 [==============================] - 0s 3ms/step - loss: 0.1331 - accuracy: 0.9809 - val_loss: 0.1248 - val_accuracy: 0.9807
Epoch 4/5
161/161 [==============================] - 0s 3ms/step - loss: 0.1048 - accuracy: 0.9873 - val_loss: 0.1037 - val_accuracy: 0.9848
Epoch 5/5
161/161 [==============================] - 0s 3ms/step - loss: 0.0878 - accuracy: 0.9893 - val_loss: 0.0904 - val_accuracy: 0.9868


In [42]:
model_B.evaluate(X_test_B, y_test_B)

OSError: File write failed: time = fri apr 19 12:39:49 2024
, filename = 'my_model_a.h5', file descriptor = 44, errno = 5, error message = 'input/output error', buf = 0x558a389453e0, total write size = 4096, bytes this sub-write = 4096, bytes actually written = 18446744073709551615, offset = 0 (file write failed: time = Fri Apr 19 12:39:49 2024
, filename = 'my_model_A.h5', file descriptor = 44, errno = 5, error message = 'Input/output error', buf = 0x558a389453e0, total write size = 4096, bytes this sub-write = 4096, bytes actually written = 18446744073709551615, offset = 0)

31/31 [==============================] - 0s 2ms/step - loss: 0.0319 - accuracy: 0.4984


Exception ignored in: 'h5py._objects.ObjectID.__dealloc__'
Traceback (most recent call last):
  File "h5py/_objects.pyx", line 199, in h5py._objects.ObjectID.__dealloc__
OSError: File write failed: time = fri apr 19 12:39:49 2024
, filename = 'my_model_a.h5', file descriptor = 44, errno = 5, error message = 'input/output error', buf = 0x558a389453e0, total write size = 4096, bytes this sub-write = 4096, bytes actually written = 18446744073709551615, offset = 0 (file write failed: time = Fri Apr 19 12:39:49 2024
, filename = 'my_model_A.h5', file descriptor = 44, errno = 5, error message = 'Input/output error', buf = 0x558a389453e0, total write size = 4096, bytes this sub-write = 4096, bytes actually written = 18446744073709551615, offset = 0)


[0.03187718987464905, 0.49844881892204285]

In [43]:
model_B_on_A.evaluate(X_test_B, y_test_B)

31/31 [==============================] - 0s 2ms/step - loss: 0.0845 - accuracy: 0.9907


[0.0845453068614006, 0.9906928539276123]